In [171]:
import pandas as pd
import random
import dask.dataframe as dd
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np



In [172]:
# ddf = dd.read_csv(timeseries_file_path)
# 
# # Randomly sample 1% of the dataset
# sample_ddf = ddf.sample(frac=0.01)
# 
# # Compute the sample, converting the Dask DataFrame to a Pandas DataFrame
# sample_df = sample_ddf.compute()
# 
# print(sample_df.head())

sample_df = dd.read_csv('/Users/fionaturner-hehlen/Desktop/EnergyDataHackdays2024/1775_15_2023.csv').compute()
sample_df.head()

,id,timestamp,value_kwh,year
0,ada3c59761422f60162dd6ea3a7033b1,2023-01-27T14:30:00.000Z,0.024,2023
1,be6dea6f3c91664b87e017bd5d5cb23f,2023-02-05T15:00:00.000Z,0.026,2023
2,ada3c59761422f60162dd6ea3a7033b1,2023-02-18T02:45:00.000Z,1.074,2023
3,28f726da1433e47d62ec7f3f35c9eb86,2023-03-04T09:15:00.000Z,0.096,2023
4,9c954122d1f4cf0240dba66706cc94dc,2023-03-08T06:00:00.000Z,0.040,2023


In [173]:
# Add month column
sample_df['timestamp'] = pd.to_datetime(sample_df['timestamp'])
sample_df['year_month'] = sample_df['timestamp'].dt.to_period('M')
sample_df

/var/folders/hh/vz7rsbq10c71_9h873rwpw7w0000gn/T/ipykernel_44801/3963185989.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  sample_df['year_month'] = sample_df['timestamp'].dt.to_period('M')


,id,timestamp,value_kwh,year,year_month
0,ada3c59761422f60162dd6ea3a7033b1,2023-01-27 14:30:00+00:00,0.024,2023,2023-01
1,be6dea6f3c91664b87e017bd5d5cb23f,2023-02-05 15:00:00+00:00,0.026,2023,2023-02
2,ada3c59761422f60162dd6ea3a7033b1,2023-02-18 02:45:00+00:00,1.074,2023,2023-02
3,28f726da1433e47d62ec7f3f35c9eb86,2023-03-04 09:15:00+00:00,0.096,2023,2023-03
4,9c954122d1f4cf0240dba66706cc94dc,2023-03-08 06:00:00+00:00,0.040,2023,2023-03
...,...,...,...,...,...
934065,75c28395e3c1adc267ae47b89839fa1d,2023-02-16 20:15:00+00:00,0.060,2023,2023-02
934066,75c28395e3c1adc267ae47b89839fa1d,2023-08-18 14:15:00+00:00,0.230,2023,2023-08
934067,75c28395e3c1adc267ae47b89839fa1d,2023-02-13 20:15:00+00:00,0.110,2023,2023-02
934068,75c28395e3c1adc267ae47b89839fa1d,2023-10-02 10:15:00+00:00,0.150,2023,2023-10


In [174]:
sample_df_sorted = sample_df.sort_values(['id', 'year_month', 'timestamp'])
sample_df_sorted

,id,timestamp,value_kwh,year,year_month
145339,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 00:00:00+00:00,0.190,2023,2023-01
376799,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 00:15:00+00:00,0.025,2023,2023-01
147313,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 00:30:00+00:00,0.651,2023,2023-01
498574,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 00:45:00+00:00,0.788,2023,2023-01
150696,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 01:00:00+00:00,1.450,2023,2023-01
...,...,...,...,...,...
33077,ffe7d1f242dfca10ae7e6b3d00761954,2023-12-31 22:45:00+00:00,0.306,2023,2023-12
35056,ffe7d1f242dfca10ae7e6b3d00761954,2023-12-31 23:00:00+00:00,0.262,2023,2023-12
619263,ffe7d1f242dfca10ae7e6b3d00761954,2023-12-31 23:15:00+00:00,0.272,2023,2023-12
851472,ffe7d1f242dfca10ae7e6b3d00761954,2023-12-31 23:30:00+00:00,0.289,2023,2023-12


In [185]:
time_delta_hours = 6
time_delta = time_delta_hours * 4 # amount of 15 minute intervals

In [186]:
df_derivative = sample_df_sorted
df_derivative['last_value'] = df_derivative.groupby(['id', 'year_month'])['value_kwh'].shift(int(time_delta))
df_derivative['demand_change_kwh'] = df_derivative['value_kwh'] - df_derivative['last_value']
df_derivative.fillna(0, inplace=True)

df_derivative

,id,timestamp,value_kwh,year,year_month,last_value,demand_change_kwh
145339,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 00:00:00+00:00,0.190,2023,2023-01,0.000,0.000
376799,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 00:15:00+00:00,0.025,2023,2023-01,0.000,0.000
147313,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 00:30:00+00:00,0.651,2023,2023-01,0.000,0.000
498574,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 00:45:00+00:00,0.788,2023,2023-01,0.000,0.000
150696,002da9d5719ff5fb0f53307ace9bc9e7,2023-01-01 01:00:00+00:00,1.450,2023,2023-01,0.000,0.000
...,...,...,...,...,...,...,...
33077,ffe7d1f242dfca10ae7e6b3d00761954,2023-12-31 22:45:00+00:00,0.306,2023,2023-12,0.385,-0.079
35056,ffe7d1f242dfca10ae7e6b3d00761954,2023-12-31 23:00:00+00:00,0.262,2023,2023-12,0.610,-0.348
619263,ffe7d1f242dfca10ae7e6b3d00761954,2023-12-31 23:15:00+00:00,0.272,2023,2023-12,0.397,-0.125
851472,ffe7d1f242dfca10ae7e6b3d00761954,2023-12-31 23:30:00+00:00,0.289,2023,2023-12,0.200,0.089


In [187]:
# def apodize(signal, window_type='hamming', alpha=2.5):
#     """
#     Apply apodization (windowing) to a signal.
# 
#     Parameters:
#         signal (np.array): The input signal to be apodized.
#         window_type (str): The type of window function to apply.
#                            Options: 'hamming', 'hanning', 'blackman', 'gaussian', 'kaiser'.
#         alpha (float): Shape parameter for the Gaussian or Kaiser window (if applicable).
# 
#     Returns:
#         np.array: The apodized signal.
#     """
# 
#     N = len(signal)  # Length of the signal
# 
#     # Select the window type
#     if window_type == 'hamming':
#         window = np.hamming(N)
#     elif window_type == 'hanning':
#         window = np.hanning(N)
#     elif window_type == 'blackman':
#         window = np.blackman(N)
#     elif window_type == 'gaussian':
#         window = np.exp(-0.5 * ((np.arange(N) - N/2) / (alpha * N/2))**2)
#     elif window_type == 'kaiser':
#         window = np.kaiser(N, alpha)
#     else:
#         raise ValueError(f"Unsupported window type: {window_type}")
# 
#     # Apply the window to the signal
#     apodized_signal = signal * window
# 
#     return apodized_signal


In [188]:
#def max_apodize(signal, window_type='hamming', alpha=0.05):
#    return apodize(signal, window_type=window_type, alpha=alpha).max()

In [189]:
#def min_apodize(signal, window_type='hamming', alpha=0.05):
#    return apodize(signal, window_type=window_type, alpha=alpha).min()

In [190]:
rolling_average = 5

In [191]:
def smooth_signal(signal, window_size=rolling_average):
    """
    Smooth the input signal using a rolling mean (moving average).
    
    Parameters:
        signal (np.array): The input signal to be smoothed.
        window_size (int): The size of the rolling window (must be odd).
    
    Returns:
        np.array: The smoothed signal.
    """
    if window_size % 2 == 0:
        raise ValueError("Window size must be odd to keep symmetry.")
    
    # Convert to pandas Series to use rolling mean
    signal_series = pd.Series(signal)
    
    # Apply rolling mean
    smoothed_signal = signal_series.rolling(window=window_size, center=True, min_periods=1).mean().to_numpy()
    
    return smoothed_signal

In [192]:
def smooth_max(signal, window_size=rolling_average):
    return smooth_signal(signal, window_size).max()

def smooth_min(signal, window_size=rolling_average):
    return smooth_signal(signal, window_size).min()

In [193]:
from scipy.signal import savgol_filter

savgol_order = 3

def max_savgol(signal, window_size=rolling_average, polyorder=savgol_order):
    return savgol_filter(signal, window_size, polyorder).max()

def min_savgol(signal, window_size=rolling_average, polyorder=savgol_order):
    return savgol_filter(signal, window_size, polyorder).min()


In [194]:
# Step 1: Group by 'id' and extract features
agg_funcs = {
    'value_kwh': ['mean', 'std', 'min', 'max'],
    'demand_change_kwh': ['mean', 'std', 'max', smooth_max, max_savgol, 'min', smooth_min, min_savgol]
}


monthly_features_df = df_derivative.groupby(['id', 'year_month']).agg(agg_funcs)

monthly_features_df

value_kwh                          \
                                                 mean       std    min    max   
id                               year_month                                     
002da9d5719ff5fb0f53307ace9bc9e7 2023-01     0.763014  0.772785  0.020  3.532   
                                 2023-02     0.680193  0.744177  0.000  3.265   
                                 2023-03     0.584596  0.705993  0.019  3.145   
                                 2023-04     0.513881  0.660658  0.016  2.940   
                                 2023-05     0.357994  0.614618  0.004  2.439   
...                                               ...       ...    ...    ...   
ffe7d1f242dfca10ae7e6b3d00761954 2023-08     0.079614  0.106065  0.014  1.319   
                                 2023-09     0.082902  0.104470  0.014  0.960   
                                 2023-10     0.089666  0.114843  0.014  1.008   
                                 2023-11     0.091762  0.105300  0.014  0.959   
                                 2023-12     0.104182  0.118623  0.026  1.135   

                                            demand_change_kwh            \
                                                         mean       std   
id                               year_month                               
002da9d5719ff5fb0f53307ace9bc9e7 2023-01            -0.002744  1.070998   
                                 2023-02            -0.006327  1.022264   
                                 2023-03            -0.006275  0.957519   
                                 2023-04            -0.002864  0.920220   
                                 2023-05            -0.004096  0.878292   
...                                                       ...       ...   
ffe7d1f242dfca10ae7e6b3d00761954 2023-08             0.000531  0.152554   
                                 2023-09             0.000681  0.147509   
                                 2023-10             0.000293  0.163829   
                                 2023-11             0.000416  0.143677   
                                 2023-12             0.001986  0.165025   

                                                                          \
                                               max smooth_max max_savgol   
id                               year_month                                
002da9d5719ff5fb0f53307ace9bc9e7 2023-01     3.234     2.2834   2.665971   
                                 2023-02     3.102     2.2744   2.612543   
                                 2023-03     2.830     1.7916   2.603029   
                                 2023-04     2.879     1.7050   2.135400   
                                 2023-05     2.420     1.7184   2.086114   
...                                            ...        ...        ...   
ffe7d1f242dfca10ae7e6b3d00761954 2023-08     1.296     0.6068   0.880571   
                                 2023-09     0.914     0.4620   0.707600   
                                 2023-10     0.958     0.5512   0.694114   
                                 2023-11     0.868     0.5070   0.765143   
                                 2023-12     1.059     0.6390   0.903229   

                                                                          
                                               min smooth_min min_savgol  
id                               year_month                               
002da9d5719ff5fb0f53307ace9bc9e7 2023-01    -3.511    -2.5710  -2.919829  
                                 2023-02    -3.153    -2.2530  -2.751429  
                                 2023-03    -3.121    -2.2964  -3.009829  
                                 2023-04    -2.920    -2.1824  -2.406486  
                                 2023-05    -2.420    -1.7476  -2.197057  
...                                            ...        ...        ...  
ffe7d1f242dfca10ae7e6b3d00761954 2023-08    -1.252    -0.5934  -0.928657  
                                 2023-09   